In [4]:
from easyAI.AI import NegamaxNoPruning
from easyAI import TwoPlayerGame, Human_Player, AI_Player, Negamax, NegamaxNoPruning
import random
from tqdm.auto import tqdm

In [5]:
class Nim(TwoPlayerGame):
    def __init__(self, players, piles = [5, 5, 5, 5], deterministic = False):
        self.piles = piles
        self.players = players
        self.nplayer = 1
        self.current_player = 1
        self.deterministic = deterministic
    
    def possible_moves(self):
        moves = ["%d,%d" % (i + 1, j) for i in range(len(self.piles)) for j in range(1, self.piles[i] + 1)]
        return moves
    
    def make_final_move(self, move):
        pile, count = move.split(",")
        extra = 1 if random.randint(1, 10) == 1 and not self.deterministic else 0
        self.piles[int(pile) - 1] -= int(count) - extra
        return extra
    
    def make_move(self, move):
        pile, count = move.split(",")
        self.piles[int(pile) - 1] -= int(count)

    def unmake_move(self, move): # optional method (speeds up the AI)
        pile, count = move.split(",")
        self.piles[int(pile) - 1] += int(count)

    def show(self): print(" ".join(map(str, self.piles)))

    def win(self): return max(self.piles) == 0

    def is_over(self): return self.win()

    def scoring(self): return 100 if self.win() else 0

    def ttentry(self): return tuple(self.piles) #optional, speeds up AI

# Negamax

In [6]:
# Win tracking dict
results = {}

depths = [3, 7]
deterministic = [True, False]
game_amount = 10

for d1, d2 in tqdm([(d1, d2) for d1 in depths for d2 in depths], desc="Processing depths"):
    for det in tqdm(deterministic, desc="Processing deterministic"):
        # print(f"depth1: {d1}, depth2: {d2} deterministic: {det}")
        
        # Play 10 games and track the results
        p1_wins = 0
        p2_wins = 0
        p1_avg_time = 0
        p2_avg_time = 0

        for x in tqdm(range(game_amount), desc="Playing games"):
            ai = Negamax(d1)
            ai2 = Negamax(d2)
            game = Nim(players=[AI_Player(ai), AI_Player(ai2)],
                    deterministic=det,
                    piles=[5,5,5,5])
            p1_time, p2_time = game.play(verbose=False)
            p1_avg_time += p1_time
            p2_avg_time += p2_time
            # print("\nplayer %d wins\n" % game.current_player)
            if game.current_player == 1:
                p1_wins += 1
            else:
                p2_wins += 1 

        p1_avg_time /= game_amount
        p2_avg_time /= game_amount
        results[(d1, d2, det)] = {"p1_wins": p1_wins,
                                    "p2_wins": p2_wins,
                                    "p1_average_game_time": p1_avg_time,
                                    "p2_average_game_time": p2_avg_time} # Update the results

print(results)

Processing depths:   0%|          | 0/4 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

...

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

{(3, 3, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.00032284061999973804, 'p2_average_game_time': 0.00032643342221629404}, (3, 3, False): {'p1_wins': 1, 'p2_wins': 9, 'p1_average_game_time': 0.00037722063863429177, 'p2_average_game_time': 0.0003784540660614073}, (3, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.00030049969999652144, 'p2_average_game_time': 0.02232468786666636}, (3, 7, False): {'p1_wins': 3, 'p2_wins': 7, 'p1_average_game_time': 0.00031671366863765695, 'p2_average_game_time': 0.02276145709454843}, (7, 3, True): {'p1_wins': 10, 'p2_wins': 0, 'p1_average_game_time': 0.028406229699998498, 'p2_average_game_time': 0.00032280886666676966}, (7, 3, False): {'p1_wins': 7, 'p2_wins': 3, 'p1_average_game_time': 0.025529068405403005, 'p2_average_game_time': 0.00029152145737249165}, (7, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.023933521789999194, 'p2_average_game_time': 0.021653484222219452}, (7, 7, False): {'p1_wins': 

## TODO:
wziac rzeczy z results i se zrobic plota jakiegos fajnego

# Negamax bez alpha-beta pruning
(zakładając, że copilot dobrze zaimplementował (ale chyba jest git xd))

In [7]:
# Win tracking dict
results = {}

depths = [3, 7]
deterministic = [True, False]
game_amount = 10

for d1, d2 in tqdm([(d1, d2) for d1 in depths for d2 in depths], desc="Processing depths"):
    for det in tqdm(deterministic, desc="Processing deterministic"):
        # print(f"depth1: {d1}, depth2: {d2} deterministic: {det}")
        
        # Play 10 games and track the results
        p1_wins = 0
        p2_wins = 0
        p1_avg_time = 0
        p2_avg_time = 0

        for x in tqdm(range(game_amount), desc="Playing games"):
            ai = NegamaxNoPruning(d1)
            ai2 = NegamaxNoPruning(d2)
            game = Nim(players=[AI_Player(ai), AI_Player(ai2)],
                    deterministic=det,
                    piles=[5,5,5,5])
            p1_time, p2_time = game.play(verbose=False)
            p1_avg_time += p1_time
            p2_avg_time += p2_time
            # print("\nplayer %d wins\n" % game.current_player)
            if game.current_player == 1:
                p1_wins += 1
            else:
                p2_wins += 1 

        p1_avg_time /= game_amount
        p2_avg_time /= game_amount
        results[(d1, d2, det)] = {"p1_wins": p1_wins,
                                    "p2_wins": p2_wins,
                                    "p1_average_game_time": p1_avg_time,
                                    "p2_average_game_time": p2_avg_time} # Update the results

print(results)

Processing depths:   0%|          | 0/4 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

...

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Processing deterministic:   0%|          | 0/2 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

Playing games:   0%|          | 0/10 [00:00<?, ?it/s]

{(3, 3, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.002352423430002091, 'p2_average_game_time': 0.002234599211113972}, (3, 3, False): {'p1_wins': 6, 'p2_wins': 4, 'p1_average_game_time': 0.002164716782654509, 'p2_average_game_time': 0.0019697112114301684}, (3, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 0.0022851577800014414, 'p2_average_game_time': 6.192900851811118}, (3, 7, False): {'p1_wins': 4, 'p2_wins': 6, 'p1_average_game_time': 0.002477701099100035, 'p2_average_game_time': 6.702792869455867}, (7, 3, True): {'p1_wins': 10, 'p2_wins': 0, 'p1_average_game_time': 9.457783687477768, 'p2_average_game_time': 0.0021668997222124946}, (7, 3, False): {'p1_wins': 8, 'p2_wins': 2, 'p1_average_game_time': 9.432899639012003, 'p2_average_game_time': 0.002168304355968528}, (7, 7, True): {'p1_wins': 0, 'p2_wins': 10, 'p1_average_game_time': 8.437451273259963, 'p2_average_game_time': 6.304128975500049}, (7, 7, False): {'p1_wins': 5, 'p2_wins': 5, 'p1_average